In [1]:
import random
from sklearn.model_selection import KFold
import math
import pandas as pd
import shutil
import time
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os

2022-12-06 10:05:33.459761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["dlopen(/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so, 0x0006): symbol not found in flat namespace '__ZN3tsl2io7DirnameENSt3__117basic_string_viewIcNS1_11char_traitsIcEEEE'"]
  warnings.warn(f"unable to load libtensorflow_io_pl

In [26]:
paths = {
    'TRAIN_PATH' : os.path.join('workspace', 'images', 'train'),
    'TEST_PATH' : os.path.join('workspace', 'images','test'),
    'EVAL_PATH' : os.path.join('workspace', 'images','eval'),
    'IMAGES_PATH': os.path.join('workspace','images','all'),
    'ANNOTATION_PATH': os.path.join('workspace','annotations'),
    'RESULTS_PATH': os.path.join('workspace','results')
 }

In [31]:
input_height = int(1920/8)
input_width = int(1080/8)
batch_size = 4

augmentation=True
# TODO: find best ones using the display_data_augmentation_sample jupyter notebook
rescale_size=1./255
rotation_range=20 # range within which we randomly rotate pictures
width_shift_range=0.05 # range withing which we shift image horizontally
height_shift_range=0.05 # range withing which we shift image vertically
shear_range=0.2 # random shearing transformation
zoom_range=0.2 # random zoom inside pictures
horizontal_flip=True
fill_mode='nearest' # strategy to fill newly created pixels after rotation or shift
# consider also stride (>1), type of pooling (Max, Avg), padding (same, valid)

In [32]:
K = 10

In [33]:
model_path = os.path.join("models","CNN_baseline.h5")

In [34]:
epochs = 2

In [35]:
def emptyFolders():
    for file in os.listdir(paths["TRAIN_PATH"]):
        if file != ".DS_Store":
            os.remove(os.path.join(paths["TRAIN_PATH"], file))
    for file in os.listdir(paths["TEST_PATH"]):
        if file != ".DS_Store":
            os.remove(os.path.join(paths["TEST_PATH"], file))

df = pd.read_csv(os.path.join(paths['ANNOTATION_PATH'],"annotations.csv"))
df = df.dropna()

In [38]:
## Training with K-fold cross validation
kf = KFold(n_splits=K, random_state=None, shuffle=True)
kf.get_n_splits(df)

df_results = pd.DataFrame()

# TODO: change adapting to our project
df_new = pd.DataFrame()
for _, row in df.iterrows():
    if(row['Path'].endswith(".jpg")):
        new_row = { 'Path': row['Path'], 'Class': row['Class'] }
        df_new_row = pd.DataFrame([new_row])
        df_new = pd.concat([df_new, df_new_row],ignore_index=True)

time.sleep(5)
i = 1
for train_index, test_index in kf.split(df_new):
    print("====== K-Fold CV Iteration: %d/%d =======" % (i,K))
    emptyFolders()
    print("Folders empty..")

    print("Train: ", len(train_index))
    print("Test: ", len(test_index))

    df_train = df_new.iloc[train_index]
    df_test = df_new.iloc[test_index]

    for _, row in df_train.iterrows():
        shutil.copy(os.path.join(paths["IMAGES_PATH"],row['Path']),
                os.path.join(paths["TRAIN_PATH"],row['Path']))
    for _, row in df_test.iterrows():
        shutil.copy(os.path.join(paths["IMAGES_PATH"],row['Path']),
                os.path.join(paths["TEST_PATH"],row['Path']))

    if augmentation:
        train_datagen = ImageDataGenerator(
          rescale=rescale_size,
          rotation_range=rotation_range,
          width_shift_range=width_shift_range,
          height_shift_range=height_shift_range,
          shear_range=shear_range,
          zoom_range=zoom_range,
          horizontal_flip=horizontal_flip,
          fill_mode=fill_mode)
    else:
        train_datagen = ImageDataGenerator(rescale=1./255)

    train_dir=paths['TRAIN_PATH']
    # TODO: Consider if the output should be normalized
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

    # validation set image data generator
    val_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented
    validation_dir=paths['TEST_PATH']
    validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

    try:
        model = load_model(model_path, compile=True)
    except Exception as OSError:
        pass

    # set steps, epochs, etc.
    history = model.fit_generator(
            train_generator,
            steps_per_epoch=int(math.ceil((1. * len(df_train)) / batch_size)),
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=int(math.ceil((1. * len(df_test)) / batch_size)))

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    for epoch in range(epochs):
        new_row = { 'k': i, 'epoch': epoch+1, 'acc': acc[epoch], 'val_acc': val_acc[epoch], 'loss': loss[epoch], 'val_loss': val_loss[epoch] }
        df_new_row = pd.DataFrame([new_row])
        df_results = pd.concat([df_results, df_new_row],ignore_index=True)

    model.save(model_path)
    i+=1

print(df_results)

====== K-Fold CV Iteration: 1/10 =======
Folders empty..
Train:  180
Test:  21
Letter to be removed from the training set: B
Train:  163
Test:  38
Created csv files..
Found 163 validated image filenames.
Found 38 validated image filenames.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_38356/3992552463.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/2
16/16 [==============================] - 9s 520ms/step - loss: 4825.6699 - mae: 63.5557 - val_loss: 5980.0312 - val_mae: 73.7494
Epoch 2/2
16/16 [==============================] - 8s 464ms/step - loss: 5967.6528 - mae: 70.6818 - val_loss: 5049.3779 - val_mae: 67.2242
====== K-Fold CV Iteration: 2/10 =======
Folders empty..
Train:  181
Test:  20
Letter to be removed from the training set: B
Train:  165
Test:  36
Created csv files..
Found 165 validated image filenames.
Found 36 validated image filenames.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_38356/3992552463.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/2
16/16 [==============================] - 7s 392ms/step - loss: 5499.6514 - mae: 68.5930 - val_loss: 4553.2134 - val_mae: 62.6491
Epoch 2/2
16/16 [==============================] - 8s 473ms/step - loss: 6233.6055 - mae: 71.5721 - val_loss: 4019.4019 - val_mae: 58.0740
====== K-Fold CV Iteration: 3/10 =======
Folders empty..
Train:  181
Test:  20
Letter to be removed from the training set: L
Train:  159
Test:  42
Created csv files..
Found 159 validated image filenames.
Found 42 validated image filenames.


...

Epoch 1/2
16/16 [==============================] - 19s 874ms/step - loss: 6037.6523 - mae: 70.7820 - val_loss: 4742.1025 - val_mae: 65.2831
Epoch 2/2
16/16 [==============================] - 12s 725ms/step - loss: 5980.7852 - mae: 70.5718 - val_loss: 4268.9482 - val_mae: 61.3288
====== K-Fold CV Iteration: 9/10 =======
Folders empty..
Train:  181
Test:  20
Letter to be removed from the training set: L
Train:  158
Test:  43
Created csv files..
Found 158 validated image filenames.
Found 43 validated image filenames.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_38356/3992552463.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/2
15/15 [==============================] - 23s 1s/step - loss: 5447.5811 - mae: 67.0884 - val_loss: 4350.3989 - val_mae: 61.6463
Epoch 2/2
15/15 [==============================] - 12s 767ms/step - loss: 5782.8496 - mae: 69.6327 - val_loss: 8194.2754 - val_mae: 83.3521
====== K-Fold CV Iteration: 10/10 =======
Folders empty..
Train:  181
Test:  20
Letter to be removed from the training set: G
Train:  161
Test:  40
Created csv files..
Found 161 validated image filenames.
Found 40 validated image filenames.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_38356/3992552463.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/2
16/16 [==============================] - 15s 803ms/step - loss: 5582.7812 - mae: 68.9950 - val_loss: 5396.5996 - val_mae: 67.6635
Epoch 2/2
16/16 [==============================] - 12s 731ms/step - loss: 5358.3306 - mae: 66.2745 - val_loss: 4428.9331 - val_mae: 62.7619
     k  epoch        mae    val_mae         loss     val_loss
0    1      1  63.555706  73.749420  4825.669922  5980.031250
1    1      2  70.681808  67.224243  5967.652832  5049.377930
2    2      1  68.593002  62.649055  5499.651367  4553.213379
3    2      2  71.572090  58.073986  6233.605469  4019.401855
4    3      1  73.487366  67.868050  6201.523438  5547.346680
5    3      2  72.659462  72.993942  6254.145508  6253.911133
6    4      1  67.795410  58.086956  5547.118164  4426.644531
7    4      2  68.700905  57.197163  5524.520996  3914.283936
8    5      1  69.021324  71.050728  5813.517578  6196.870117
9    5      2  72.663170  69.807777  6315.198730  5416.614258
10   6      1  65.829971  64.529343  5

In [37]:
df_results.to_csv(os.path.join(paths['RESULTS_PATH'], "resultsKFold.csv"), index=False)